# Initialize the environment

* Loaded necessary system-wide packages

In [1]:
// basic docs for C# interactive is at: https://github.com/dotnet/interactive/tree/main/docs 
// Use XPlot for visualization, see doc at: https://fslab.org/XPlot/
// For several examples of data analysis methods see: https://xamlbrewer.wordpress.com/category/ml-net/
#r "nuget:XPlot.Plotly"
#r "nuget:Microsoft.ML"
#r "nuget:Microsoft.Data.Analysis,0.4.0"
#r "nuget:Microsoft.DotNet.Interactive.ExtensionLab,*-*"
using System;
using System.Threading;
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.Data;
using Microsoft.Data.Analysis; 
using XPlot.Plotly;

/*
var dateRef = DateTime.Parse("2020/12/01 12:00:00");
var dateSample = DateTime.Parse("2020/12/07 12:47:00");
(dateSample-dateRef).TotalSeconds
*/

521220

Load IcsMonitor and process the input file. 

In [1]:
#r "C:\GitHub\traffix.net\samples\IcsMonitor\bin\Debug\netcoreapp3.1\IcsMonitor.dll"
using IcsMonitor;
var ctx = new Interactive();

var reader = ctx.CreateCaptureFileReader(@"G:\Shared drives\BONNET\AD Metody\Modbus\ModbusFactory\Assembler\Net-Assembler_Basic_Normal.cap");
var frames  = ctx.GetNextFrames(reader).ToList();
var firstFrame = frames.First();
var timeBaseTicks = firstFrame.Ticks;
Console.WriteLine($"Frames={frames.Count}");

var tables = ctx.PopulateConversationTables(frames, @"c:\temp\assembler_basic_normal.fc\", TimeSpan.FromMinutes(1), CancellationToken.None).ToList();
Console.WriteLine($"Tables={tables.Count}");

var processor = new IcsMonitor.Modbus.ModbusBiflowProcessor();
var conversations = tables.SelectMany(x=> ctx.GetConversations(x,processor)).ToList();
Console.WriteLine($"Conversations={conversations.Count}");

reader.Close();
var dataFrame = conversations.ToDataFrame();
#!linqify dataFrame


Frames=984613


Tables=30


Conversations=30


## Ratio of operations in the flow
The following graph shows the ratio of different operations within the flow. The ratio is computed as the number of the corresponding operations to the total number of packets in the flow.  

In [1]:
var writeSingleCoilRequests = conversations.Select(x=>(packets:x.ForwardMetrics.Packets, value: (float)x.Data.WriteSingleCoilRequests/x.ForwardMetrics.Packets));
var readDiscreteInputsRequests = conversations.Select(x=>(packets:x.ForwardMetrics.Packets, value: (float)x.Data.ReadDiscreteInputsRequests/x.ForwardMetrics.Packets));

var plot = Chart.Plot(new []{ 
    new Graph.Scatter { x = writeSingleCoilRequests.Select(x=>x.packets), y = writeSingleCoilRequests.Select(x=>x.value), mode = "markers", name="WriteSingleCoilRequests" },
    new Graph.Scatter { x = readDiscreteInputsRequests.Select(x=>x.packets), y = readDiscreteInputsRequests.Select(x=>x.value), mode = "markers", name="ReadDiscreteInputsRequests" }
});
plot.WithXTitle("Packets");
plot.WithYTitle("Operation");
plot.WithTitle("Operations Ratio Graph");
plot

# Basic Packet-level Stats
This section shows various basic statistics of the capture. 

In [1]:
using Traffix.Providers.PcapFile;
using Traffix.Extensions.Decoders.Industrial;
var packets = frames.Select(x=> (Time: new TimeSpan(x.Ticks - timeBaseTicks).TotalSeconds, Packet: x.GetTcpPacket())).Where(x=>x.Packet is not null && x.Packet.PayloadData?.Length > 0).ToList();

var requests = packets.Where(x=>x.Packet.DestinationPort == 502);
var responses = packets.Where(x=>x.Packet.SourcePort == 502);

var modbusRequests = requests.Select(x=> (x.Time, Modbus: x.Packet.DecodeOrDefault(stream => new ModbusRequestPacket(stream)))).Where(x=>x.Modbus is not null);
var modbusFunctions = modbusRequests.Select(m =>  (Time: m.Time, Code: m.Modbus.Function.ToString()));

## Function count
The following plot provides information on the total number of individual function codes in the capture. 

In [1]:
var modbusGroupByFunctions = modbusFunctions.GroupBy(x=> x.Code);
var plot = Chart.Plot( new []{ 
        new Graph.Bar { x = modbusGroupByFunctions.Select(x=>x.Key), y = modbusGroupByFunctions.Select(x=>x.Count()), name="functions" }
    });
plot.WithXTitle("Fuctions");
plot.WithYTitle("Count");
plot.WithTitle("Functions Count");

plot

# Timeline

The present section visualizes the traffic in terms of various timelines. 

## Message timeline
The basic form of timeline shows a number of non-empty TCP packets in both directions. Although the communication follows the regular pattern the timeline communication diagram shows the small variance in the number of packets as caused by system and network load. 

In [1]:
var requestsAggregation = requests.GroupBy(x=> (int)(x.Time/10));
var responsesAggregation = responses.GroupBy(x=> (int)(x.Time/10));
var plot = Chart.Plot( new []{ 
    new Graph.Scatter { x = requestsAggregation.Select(x=>x.Key), y = requestsAggregation.Select(x=>x.Count()), mode = "line", name="requests" },
    new Graph.Scatter { x = responsesAggregation.Select(x=>x.Key), y = responsesAggregation.Select(x=>x.Count()), mode = "line", name="responses" }
});
plot.WithXTitle("Time");
plot.WithTitle("Message Timeline");
plot.Width = 1000;
plot.Height = 300;

plot

## Function code timeline
In order to see the pattern of the system, the MODBUS function requests are extracted from packets and shown in the next graph. Four MODBUS functions are observed:

* ReadHoldingRegister
* ReadInputStatus
* WriteSingleRegister
* WriteSIngleCoil


In [1]:
var functionsPrefix = modbusFunctions.TakeWhile(x=>x.Time<1);

var g = new Graph.Scatter { x = functionsPrefix.Select(x=>x.Time), y = functionsPrefix.Select(x=>x.Code), mode = "markers", name="function sequence" };
var plot = Chart.Plot( new []{ g });
plot.WithXTitle("Time");
plot.WithTitle("Function Timeline");
plot.Width = 1000;
plot.Height = 300;

plot
